In [1]:
import os
import random
import pickle
import numpy as np
from music21 import converter, instrument, note, chord
from sklearn.model_selection import train_test_split

# Misc
import pickle

# Data manipulation
import numpy
import pandas as pd

# Music
import music21 as m21
from music21 import converter, instrument, note, chord

# Data Visualiation
import matplotlib.pyplot as plt
import seaborn as sns

# System
import os
import random
import shutil

In [4]:
source_dir = "../../data_raw/"

# Define the percentage of data to use to training the modle 
train_percentage = 80  


train_dir = "../../data_split/train/"
test_dir = "../../data_split/test/"


os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)


all_files = os.listdir(source_dir)

num_train_files = int(len(all_files) * (train_percentage / 100))


random.shuffle(all_files)

# Split the files into training and testing sets
train_files = all_files[:num_train_files]
test_files = all_files[num_train_files:]


for file in train_files:
    source_file = os.path.join(source_dir, file)
    destination_file = os.path.join(train_dir, file)
    shutil.copy(source_file, destination_file)


for file in test_files:
    source_file = os.path.join(source_dir, file)
    destination_file = os.path.join(test_dir, file)
    shutil.copy(source_file, destination_file)

In [5]:
def load_midi_into_notes():
    # Extracted notes for each file
    notes = []

    # Save the indexes in notes in which a new composition started to create consistent sequences
    new_composition_indexes = []

    # Load THE !!TRAINING!! MIDI files to create the vocabulary
    input_path = "../../data_split/test"
    for file in os.listdir(input_path):
        file_path = os.path.join(input_path, file)
        
        # Add new index to alert that it's a new composition
        if new_composition_indexes and new_composition_indexes[-1] != len(notes):
            new_composition_indexes.append(len(notes))

        try:
            # Convert the music into a score object
            score = converter.parse(file_path)

            print("Parsing %s" % file)

            elements_in_part = None

            try:  # File has instrument parts
                # Given a score that represents the MIDI, partition it into parts for each unique instrument found
                partitions_by_instrument = instrument.partitionByInstrument(score)
                # Visit all the elements (notes, chords, rests, and more) of each of its internal "measures."
                elements_in_part = partitions_by_instrument.parts[0].recurse()

            except:  # File has notes in a flat structure
                elements_in_part = score.flat.notes

            # Scroll through all the elements (notes or chords) picked up
            for element in elements_in_part:
                # If the element is a note...
                if isinstance(element, note.Note):
                    # Add note to array
                    notes.append(str(element.pitch))
                # If the element is a chord (a set of notes --> e.g., C4 F4)
                elif isinstance(element, chord.Chord):
                    # Extract each note from the chord and insert it into the array in the format Note1.Note2.Note3
                    notes.append('.'.join(str(n) for n in element.normalOrder))

        except Exception as e:
            print(f"Error parsing {file}: {str(e)}")
            continue

    print("✅Loading training done")

    # Save the 'notes' list to a pickle file
    os.makedirs("../../data_vocab/", exist_ok=True)
    with open("../../data_vocab/notes.pkl", "wb") as filepath:
        pickle.dump(notes, filepath)

    # Return notes and new composition indexes
    return notes, new_composition_indexes


In [8]:
notes_train, new_comp_indexes_train = load_midi_into_notes()


sequence_length = 100


generated_notes = []


# Loop through the notes in the training data
for i in range(len(notes_train) - sequence_length):
    # Get the last note or chord in the sequence
    last_note = notes_train[i + sequence_length - 1]
    
   
    generated_notes.append(last_note)

# Save the generated sequence to a MIDI file
output_midi_filename = os.makedirs(""../../output_baseline", exist_ok=True)
output_midi = converter.parse(output_midi_filename)

for note_str in generated_notes:
    if '.' in note_str:
        # chord
        notes_in_chord = note_str.split('.')
        chord_notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(current_note)
            new_note.storedInstrument = instrument.Piano()
            chord_notes.append(new_note)
        new_chord = chord.Chord(chord_notes)
        new_chord.offset = len(output_midi.elements)
        output_midi.insert(new_chord)
    else:
        #  note
        new_note = note.Note(note_str)
        new_note.storedInstrument = instrument.Piano()
        new_note.offset = len(output_midi.elements)
        output_midi.insert(new_note)

output_midi.show()
output_midi.write('midi', fp='baseline_generated_music.midi')

Parsing mazrka35.mid
Parsing chopol14.mid
Parsing mk_chim2.mid
Parsing chpnimpu.mid
Parsing chopineb.mid
Parsing chetude1.mid
Parsing mazrka22.mid
Parsing chopol15.mid
Error parsing lecpsc3.mid: badly formatted midi bytes, got: b'RIFF\x0eN\x00\x00RMIDdata\x01N\x00\x00'
Parsing mazrka12.mid
Parsing choval13.mid
Parsing chopol05.mid
Parsing pologm.mid
Parsing fugaam.mid
Parsing prelop45.mid
Parsing nocturne.mid
Parsing mazrka28.mid
Parsing op72.mid
Parsing mazrka03.mid
Parsing chonoc08.mid
Parsing souvpaga.mid
Parsing chonoc12.mid
Parsing lecpsd3.mid
Parsing choschz2.mid
Parsing chp4iapb.mid
Parsing chonoc21.mid
Parsing mazrka16.mid
Parsing mazrka37.mid
Parsing mazrka07.mid
Parsing mazrka36.mid
Parsing choval11.mid
Parsing chonoc15.mid
Parsing chopin-1.mid
Parsing mazrka24.mid
Parsing chopol16.mid
Parsing mazrka34.mid
Parsing ballade3.mid
Parsing chopol09.mid
Parsing choval17.mid
Parsing chpnbarc.mid
Parsing choval16.mid
Parsing nouv3.mid
Parsing chonoc11.mid
Parsing mazrka30.mid
Parsing

FileNotFoundError: Cannot find file in _generated_music.midi